# Mount Google Drive

# Utilities

In [2]:
def count_label_occurances(label_list):
  print('Counting occurrences of target classes:')
  label_counts = pd.DataFrame(label_list, columns=['digit'])['digit'].value_counts()
  label_percentages = (label_counts / label_counts.sum()) * 100

  label_summary = pd.DataFrame({
      'Count': label_counts,
      'Percentage': label_percentages
  }).sort_index()

  print(label_summary)

In [3]:
def generate_label_index_correspondance(images, labels):
  images_and_labels = {}
  for i in range (0, len(images)):
    images_and_labels[i] = labels[i]
  return images_and_labels

In [4]:
def print_shape(images, labels):
  print("Shape of images: ", images.shape)
  print("Shape of labels: ", labels.shape)

In [5]:
def display_images_with_labels(images, labels, num_img=30, images_per_row=5, indices=None, random_subset=True):
    if random_subset:
        selected_indices = np.random.choice(range(len(images)), size=num_img, replace=False)
    else:
        if indices is None:
            raise ValueError("Indices must be provided if random_subset is set to False.")
        selected_indices = indices

    num_rows = (len(selected_indices) + images_per_row - 1) // images_per_row
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(images_per_row * 3, num_rows * 3))
    axes = np.array(axes).reshape(-1)

    for i, idx in enumerate(selected_indices):
        ax = axes[i]
        img = images[idx]
        img = np.squeeze(img)
        ax.imshow(img, vmin=0., vmax=255.)
        ax.set_title(f'Label: {labels[idx]}')
        ax.axis('off')

    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

In [6]:
def remove_instances(images, labels, to_be_removed):

    # Print initial shape
    print("Shape before: ", images.shape, labels.shape)

    # Remove duplicates
    images_unique = np.delete(images, to_be_removed, axis=0)
    labels_unique = np.delete(labels, to_be_removed, axis=0)

    # Print resulting shape
    print("Shape after: ", images_unique.shape, labels_unique.shape)

    # Calculate and print the number of removed duplicates
    num_removed = images.shape[0] - images_unique.shape[0]
    print(f"We removed {num_removed} images.")

    return images_unique, labels_unique

In [7]:
def find_class_indices(labels):
    class_indices = {}
    for class_id in np.unique(labels):
        class_indices[class_id] = np.where(labels == class_id)[0]
    return class_indices

In [8]:

def select_random_subset(input_list, subset_size):
    if subset_size > len(input_list):
        raise ValueError("Subset size cannot be greater than the size of the input list.")

    indices = np.random.choice(len(input_list), subset_size, replace=False)
    return [input_list[i] for i in indices]

In [9]:
def reshuffle(images, labels):
    assert len(images) == len(labels), "Images and labels arrays must have the same length."
    rng = np.random.default_rng()
    indices = rng.permutation(len(images))
    images = images[indices]
    labels = labels[indices]
    return images, labels

#Initialization and data fetching

In [10]:
# Install for access to RandAugment

!pip install keras-cv --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 37.5 MB/s eta 0:00:00


In [11]:
# Miscellaneous imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib
from PIL import Image, ImageEnhance
import cv2
import gc


# SciKit imports
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight


# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from keras_cv.layers import RandAugment
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import LossScaleOptimizer


# Model-specific imports
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_mobilenet

print(tf.__version__)

# Define random seed for reprodusability
seed = 69
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

2.17.1


In [12]:
# Import dataset and define integer-name correspondance for convenience when displaying images

data = np.load('training_set.npz')
X = data['images']
y = data['labels']


labels = {
0: 'Basophil',
1: 'Eosinophil',
2: 'Erythroblast',
3: 'Immature granulocytes',
4: 'Lymphocyte',
5: 'Monocyte',
6: 'Neutrophil',
7: 'Platelet',
}
unique_labels = list(labels.values())

# Removing duplicate images

In [13]:
# Locate the duplicate images

def find_duplicate_images(images, labels):
    hashes = {}
    duplicates = {}

    for i, img in enumerate(images):
        img_flat = img.tobytes()
        img_hash = hashlib.md5(img_flat).hexdigest()

        if img_hash in hashes:
            first_index = hashes[img_hash]
            duplicates[i] = {'label': labels[i], 'first_index': first_index}
        else:
            hashes[img_hash] = i  #

    return duplicates

duplicates = find_duplicate_images(X, y)
to_be_removed = list(duplicates.keys())
print(len(duplicates.keys()), " duplicates located.")

1806  duplicates located.


In [14]:
# to_be_removed contains the indices at which you will find the duplicates.
# For the legitimate images no more needs to be done, but as we do not want ANY copies
# of the illegitimate images (Rick Astley and astonaut(?)), we will manually append
# the indices at which these first occur

to_be_removed.append(13559)
to_be_removed.append(11959)

X_unique, y_unique = remove_instances(X, y, to_be_removed)


Shape before:  (13759, 96, 96, 3) (13759, 1)
Shape after:  (11951, 96, 96, 3) (11951, 1)
We removed 1808 images.


In [15]:
# NO IMPORTANT FUNCTIONALITY
# Just a check that the image-label correspondance is still correct (it is)

label_index_correspondance = generate_label_index_correspondance(X, y)
unique_label_index_correspondance = generate_label_index_correspondance(X_unique, y_unique)


print("3440 (label: 1) is removed:")
for i in range(3437,3443):
  print("Index:", i, "label: ", label_index_correspondance[i])

print("\n")
for i in range(3437,3443):
  print("Index:", i, "label: ", unique_label_index_correspondance[i])
#print(label_index_correspondance[3437:3443])

print("\n")
print("3440 (label: 1) and 4761 (label: 3) is removed:")
for i in range(4758,4764):
  print("Index:", i, "label: ", label_index_correspondance[i])

print("\n")
for i in range(4758,4764):
  print("Index:", i, "label: ", unique_label_index_correspondance[i])

# Garbage collection
del label_index_correspondance
del unique_label_index_correspondance
del X
del y
gc.collect()

3440 (label: 1) is removed:
Index: 3437 label:  [5]
Index: 3438 label:  [1]
Index: 3439 label:  [5]
Index: 3440 label:  [1]
Index: 3441 label:  [5]
Index: 3442 label:  [7]


Index: 3437 label:  [5]
Index: 3438 label:  [1]
Index: 3439 label:  [5]
Index: 3440 label:  [5]
Index: 3441 label:  [7]
Index: 3442 label:  [5]


3440 (label: 1) and 4761 (label: 3) is removed:
Index: 4758 label:  [4]
Index: 4759 label:  [3]
Index: 4760 label:  [0]
Index: 4761 label:  [3]
Index: 4762 label:  [6]
Index: 4763 label:  [2]


Index: 4758 label:  [3]
Index: 4759 label:  [0]
Index: 4760 label:  [6]
Index: 4761 label:  [2]
Index: 4762 label:  [5]
Index: 4763 label:  [5]


0

# Splitting into validation and training sets


In [16]:
# Printing the label distribution for the dataset where duplicates have been removed
# (for later reference)

count_label_occurances(y_unique)

Counting occurrences of target classes:
       Count  Percentage
digit                   
0        850    7.112376
1       2179   18.232784
2       1085    9.078738
3       2023   16.927454
4        849    7.104008
5        992    8.300561
6       2330   19.496276
7       1643   13.747804


In [17]:
# Creates a map class_indices where the keys are the labels as integers,
# and the values are NumPy arrays containing the indices corresponding to
# the images that belong to that label.

class_indices = find_class_indices(y_unique)

# To illustrate structure:
class_sample_indices = {cls: indices[:5] for cls, indices in class_indices.items()}
print("Class sample indices (first 5 per class):")
for cls, indices in class_sample_indices.items():
    print(f"Class {cls}: {indices[:5]}")

Class sample indices (first 5 per class):
Class 0: [26 38 42 47 79]
Class 1: [10 11 19 21 32]
Class 2: [ 6  7  9 17 31]
Class 3: [ 1 15 30 41 46]
Class 4: [28 37 39 52 53]
Class 5: [22 23 29 45 49]
Class 6: [ 2  3  5 12 14]
Class 7: [ 0  4  8 13 16]


In [18]:
# Set aside 80 % of data set for use as a validation set
train_end_idx = int(0.8 * len(X_unique))

# Calculate X where X is the amount of individual images from each label
# to be used in the validation set - we want to ensure equal representation
# from each label in the validation set
instances_from_each_label_for_val = (len(X_unique) - train_end_idx)//8
print(instances_from_each_label_for_val)

298


In [19]:
# For each label, we randomly choose instances_from_each_label_for_val images to be included in the validation set
# The chosen image-label pairs are subsequently removed from the training set

validation_images = []
validation_lables = []
relevant_indices_and_labels = {}

for relevant_label in range(0, len(unique_labels)):
    relevant_indices = class_indices[relevant_label]

    random_indices = select_random_subset(relevant_indices, instances_from_each_label_for_val)

    for i in random_indices:
        validation_images.append(X_unique[i])
        validation_lables.append([relevant_label])
        relevant_indices_and_labels[i] = relevant_label


to_be_removed = list(relevant_indices_and_labels.keys())

X_val_raw = np.array(validation_images)
y_val_raw = np.array(validation_lables)
X_val_raw, y_val_raw = reshuffle(X_val_raw, y_val_raw)

print("Shape of the validation set: \n")
print_shape(X_val_raw, y_val_raw)

print("Shape of the training set, before and after: \n")
X_train_raw, y_train_raw = remove_instances(X_unique, y_unique, to_be_removed)

# Garbage collection
del X_unique
del y_unique
del to_be_removed
del class_indices
del class_sample_indices
del train_end_idx
del instances_from_each_label_for_val
gc.collect()

Shape of the validation set: 

Shape of images:  (2384, 96, 96, 3)
Shape of labels:  (2384, 1)
Shape of the training set, before and after: 

Shape before:  (11951, 96, 96, 3) (11951, 1)
Shape after:  (9567, 96, 96, 3) (9567, 1)
We removed 2384 images.


0

In [20]:
# Ensure logic was sound, and that we now have an equal label representation in validation set

count_label_occurances(y_val_raw)

Counting occurrences of target classes:
       Count  Percentage
digit                   
0        298        12.5
1        298        12.5
2        298        12.5
3        298        12.5
4        298        12.5
5        298        12.5
6        298        12.5
7        298        12.5


# Augmenting the images

In [21]:
# Update class_indices to correspond to the new training set (after split)
class_indices = find_class_indices(y_train_raw)

In [22]:
# Augment data
def augment_data(X, Y, augmenter):
    X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
    augmented_images = augmenter(X_tensor)
    augmented_images_np = augmented_images.numpy()

    # We augment the images "in place" and add no new ones, so Y is just passed through
    return augmented_images_np, Y

In [23]:
# Define the augmenter to be passed to the augment_data function, dictating the intensity and
# amount per image of augmentations.

# Here, we defined three main levels for ourselves:
# 1. Baseline augmentation: augmentations_per_image=1, magnitude=0.2
  # -> Used in the main dataset for finding our baseline model (see report)

# 2. Low augmentation: augmentations_per_image=1, magnitude=0.35
# 3. Medium augmentation: augmentations_per_image=2, magnitude=0.45
# 4. High augmentation: augmentations_per_image=2, magnitude=0.6

# We experimented with all of these levels, see report for result.

rand_augment = RandAugment(
    value_range=(0, 255),
    augmentations_per_image=2,
    magnitude=0.45
)

In [24]:
# Augment the training set
X_train_augmented, y_train_augmented = augment_data(X_train_raw, y_train_raw, rand_augment)

In [25]:
# Augment the validation set, to hopefully closer resemble the hidden test set
X_val_augmented, y_val_augmented = augment_data(X_val_raw, y_val_raw, rand_augment)

In [26]:
# Ensure no funny business
print("Training set shape: ")
print_shape(X_train_augmented, y_train_augmented)
print("Validation set shape: ")
print_shape(X_val_augmented, y_val_augmented)

# Garbage collection
del X_train_raw
del y_train_raw
del rand_augment
del X_val_raw
del y_val_raw
gc.collect()

Training set shape: 
Shape of images:  (9567, 96, 96, 3)
Shape of labels:  (9567, 1)
Validation set shape: 
Shape of images:  (2384, 96, 96, 3)
Shape of labels:  (2384, 1)


8859

# Further augmentation: increasing data set size by rotating and flipping images

In [27]:
def rotate_image_90(image, amount_of_rotations=1):
    return np.rot90(image, k=amount_of_rotations, axes=(0, 1))

In [28]:
# Rotate each of the augmented images three times, so that "each image becomes four"

# Input images and labels
images = X_train_augmented
labels = y_train_augmented

class_indices = find_class_indices(labels)

images_list = list(images)
labels_list = list(labels)

for relevant_label in range(0, len(unique_labels)):
    relevant_indices = class_indices[relevant_label]
    relevant_images = []

    # Collect all images for the current label
    for i in relevant_indices:
        relevant_images.append(images[i])

    rotations = 3 # We want images at 0, 90, 180, 270 degrees
    for image in relevant_images:
      for _ in range(rotations):
          rotated_image = rotate_image_90(image)

          # To avoid "layers" of same label
          insert_position = np.random.randint(0, len(images_list) + 1)
          images_list.insert(insert_position, rotated_image)
          labels_list.insert(insert_position, [relevant_label])


# Convert back to NumPy arrays
X_train_rotated = np.array(images_list)
y_train_rotated = np.array(labels_list)

# Ensure the dataset quadrupled in size
print_shape(X_train_rotated, y_train_rotated)

# Garbage collection
del images
del labels
del X_train_augmented
del y_train_augmented
del class_indices
gc.collect()

Shape of images:  (38268, 96, 96, 3)
Shape of labels:  (38268, 1)


861

In [29]:
# To each of the rotated images, apply a flip about the y_axis.
# Append the new image to the dataset, along with its corresponding label

def augment_with_flipping(images, labels):
    images_list = list(images)
    labels_list = list(labels)

    for image, label in zip(images, labels):
        # Flip horizontally
        flipped_h = np.flip(image, axis=1)
        images_list.append(flipped_h)
        labels_list.append(label)
    return np.array(images_list), np.array(labels_list)

X_train_flipped, y_train_flipped = augment_with_flipping(X_train_rotated, y_train_rotated)

# Shuffle dataset
reshuffle(X_train_flipped, y_train_flipped)

# Ensure the dataset doubled in size
print_shape(X_train_flipped, y_train_flipped)

# Garbage collection
del X_train_rotated
del y_train_rotated
gc.collect()

Shape of images:  (76536, 96, 96, 3)
Shape of labels:  (76536, 1)


0

In [30]:
# Rename for convenience and clarity in later stages
X_train = X_train_flipped
y_train = y_train_flipped
X_val = X_val_augmented
y_val = y_val_augmented

print("Training set shape: ")
print_shape(X_train, y_train)
print("\n")
print("Validation set shape: ")
print_shape(X_val, y_val)

# Garbage collection
del X_train_flipped
del y_train_flipped
del X_val_augmented
del y_val_augmented
gc.collect()

Training set shape: 
Shape of images:  (76536, 96, 96, 3)
Shape of labels:  (76536, 1)


Validation set shape: 
Shape of images:  (2384, 96, 96, 3)
Shape of labels:  (2384, 1)


0

# Training models

## ResNet50


In [31]:
def preprocess_images_in_batches_resnet(images, batch_size=1024):
    # Placeholder for the preprocessed dataset
    preprocessed_images = np.empty_like(images, dtype=np.float32)

    # Calculate number of batches
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(images))

        # Preprocess the current batch
        batch = images[start:end].astype('float32')  # Ensure float32 for preprocessing
        preprocessed_images[start:end] = preprocess_resnet(batch)

        # Free up memory by deleting the batch (not strictly necessary in Python)
        del batch

    return preprocessed_images

In [32]:
# Scale input to ResNet-specific format
X_train_RN = preprocess_images_in_batches_resnet(X_train)
X_val_RN = preprocess_images_in_batches_resnet(X_val)

In [33]:
# Make label tensors one-hot encoded
y_train_RN = to_categorical(y_train)
y_val_RN = to_categorical(y_val)

In [ ]:
# Input shape for the model
input_shape_RN = X_train_RN.shape[1:]

# Output shape for the model
output_shape_RN = y_train_RN.shape[1]

print("Input Shape:", input_shape_RN)
print("Output Shape:", output_shape_RN)

Input Shape: (96, 96, 3)
Output Shape: 8


In [35]:
# Set training parameters

# Number of training epochs
epochs_RN = 1000

# Batch size for training
batch_size_RN = 32

# Learning rate: step size for updating the model's weights
learning_rate_RN = 0.00008

# Print the defined parameters
print("Epochs:", epochs_RN)
print("Batch Size:", batch_size_RN)
print("Learning Rate:", learning_rate_RN)

Epochs: 1000
Batch Size: 32
Learning Rate: 8e-05


In [36]:
# Calculate appropriate class weights to make up for dataset imbalance

classes = np.unique(np.argmax(y_train_RN, axis=1))  # Get unique class labels from one-hot encoded y_train
class_weights = compute_class_weight('balanced', classes=classes, y=np.argmax(y_train_RN, axis=1))
class_weight_dict_RN = dict(enumerate(class_weights))
print("Class Weights:", class_weight_dict_RN)

Class Weights: {0: 2.166440217391304, 1: 0.6357655502392344, 2: 1.519536213468869, 3: 0.6932608695652174, 4: 2.170372050816697, 5: 1.7231628242074928, 6: 0.5885211614173228, 7: 0.8891263940520446}


In [39]:
# Define the patience value for early stopping
patience_RN = 5

# Create an EarlyStopping callback
early_stopping_RN = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience_RN,
    restore_best_weights=True
)

# Store the callback in a list
callbacks_RN = [early_stopping_RN]

### With no fine tuning


In [37]:
# Non fine-tunable implementation of ResNet50
# Not our flagship model, but the one used to decide on a baseline-model ref. report

def build_resnet(
    input_shape=input_shape_RN,
    output_shape=output_shape_RN,
    learning_rate=learning_rate_RN,
    seed=seed
):
    tf.random.set_seed(seed)

    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    base_model = ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = tf.keras.layers.Input(shape=input_shape, name='Input')
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = tf.keras.layers.Dense(128, activation='relu', name='dense_1')(x)
    x = tf.keras.layers.Dropout(0.3, seed=seed, name='dropout_1')(x)
    outputs = tf.keras.layers.Dense(units=output_shape, activation='softmax', dtype='float32', name='output')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='ResNet50')

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    mixed_precision_optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

    model.compile(loss='categorical_crossentropy', optimizer=mixed_precision_optimizer, metrics=['accuracy', 'precision', 'recall'])

    return model

In [38]:
#NB! Change call to construct the non fine-tunable ResNet-model if you wish
# -> And don't forget to adjust the learning rate accordingly!

RN = build_resnet()
#RN = build_fine_tuned_resnet()
RN.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 96, 96, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 3, 3, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_avg_pool                      │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_1 (Cast)                        │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,851,016 (90.98 MB)

 Trainable params: 263,304 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [40]:
# ResNet training (NO FINE TUNING - ignore the file name)
# Trained on the baseline level of augmentation, mimicking the test we did to choose our baseline model

# Train the model with early stopping callback
history = RN.fit(
    x=X_train_RN,
    y=y_train_RN,
    batch_size=batch_size_RN,
    epochs=epochs_RN,
    validation_data=(X_val_RN, y_val_RN),
    callbacks=callbacks_RN,
    class_weight=class_weight_dict_RN
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

model_filename = 'ResNetTrulyFineTuned_forShow'+str(final_val_accuracy)+'.keras'
RN.save(model_filename)
print("Saved model as ", model_filename)

# Delete the model to free up resources
del RN

Epoch 1/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 54s 16ms/step - accuracy: 0.5661 - loss: 1.2975 - precision: 0.6874 - recall: 0.4347 - val_accuracy: 0.8108 - val_loss: 0.5420 - val_precision: 0.8636 - val_recall: 0.7567
Epoch 2/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - accuracy: 0.8270 - loss: 0.4794 - precision: 0.8754 - recall: 0.7730 - val_accuracy: 0.8523 - val_loss: 0.4461 - val_precision: 0.8907 - val_recall: 0.8201
Epoch 3/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - accuracy: 0.8678 - loss: 0.3555 - precision: 0.9023 - recall: 0.8332 - val_accuracy: 0.8683 - val_loss: 0.4117 - val_precision: 0.8926 - val_recall: 0.8402
Epoch 4/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - accuracy: 0.8954 - loss: 0.2842 - precision: 0.9206 - recall: 0.8692 - val_accuracy: 0.8800 - val_loss: 0.3903 - val_precision: 0.8982 - val_recall: 0.8582
Epoch 5/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - accuracy: 0.9132 - loss: 0.2340 - precision: 0.9320 - recall: 0.8930 - val_a

### With fine tuning (our best model)

In [37]:
# Fine-tunable implementation of ResNet50

def build_fine_tuned_resnet(
    input_shape=input_shape_RN,
    output_shape=output_shape_RN,
    base_learning_rate=learning_rate_RN,
    seed=seed
):
    tf.random.set_seed(seed)

    # Enable mixed precision
    set_global_policy('mixed_float16')
    print("Mixed precision policy set to: 'mixed_float16'")

    # Load ResNet-50 as the base model
    base_model = ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = True

    # Defining classification head - GAP, one dense layer, ReLu and dropout prob. = 0.3
    inputs = tfkl.Input(shape=input_shape, name='Input')
    x = base_model(inputs, training=True)
    x = tfkl.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = tfkl.Dense(128, activation='relu', name='dense_1')(x)  # Directly apply activation here
    x = tfkl.Dropout(0.3, seed=seed, name='dropout_1')(x)
    outputs = tfkl.Dense(units=output_shape, activation='softmax', dtype='float32', name='output')(x)


    # When we experimented with Batch Normalization, the classification head in stead looked like this:

    #inputs = tf.keras.layers.Input(shape=input_shape, name='Input')
    #x = base_model(inputs, training=False)
    #x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    #x = tf.keras.layers.Dense(128, activation=None, name='dense_1')(x)  # Remove activation here
    #x = tf.keras.layers.BatchNormalization(name='batch_norm_1')(x)       # Add BatchNorm
    #x = tf.keras.layers.Activation('relu', name='relu_activation_1')(x) # Add activation after BatchNorm
    #x = tf.keras.layers.Dropout(0.3, seed=seed, name='dropout_1')(x)
    #outputs = tf.keras.layers.Dense(units=output_shape, activation='softmax', name='output')(x)
    # <- NB! With BatchNorm, remember to increase the batch size!

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='ResNet50_FineTuned')

    # For mixed precision compatibility
    adam_optimizer = Adam(learning_rate=base_learning_rate)
    mixed_precision_optimizer = LossScaleOptimizer(adam_optimizer)

    model.compile(loss='categorical_crossentropy', optimizer=mixed_precision_optimizer, metrics=['accuracy', 'precision', 'recall'])
    return model

In [38]:

#RN = build_resnet()
RN = build_fine_tuned_resnet()
RN.summary()

Mixed precision policy set to: 'mixed_float16'
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "ResNet50_FineTuned"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 96, 96, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 3, 3, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_avg_pool                      │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_1 (Cast)                        │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,851,016 (90.98 MB)

 Trainable params: 23,797,896 (90.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [40]:
# ResNet training WITH FINE TUNING
# Trained on the medium augmentation level, like the model that performed best on the test set

# Train the model with early stopping callback
history = RN.fit(
    x=X_train_RN,
    y=y_train_RN,
    batch_size=batch_size_RN,
    epochs=epochs_RN,
    validation_data=(X_val_RN, y_val_RN),
    callbacks=callbacks_RN,
    class_weight=class_weight_dict_RN
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

model_filename = 'ResNetTrulyFineTuned_forShow'+str(final_val_accuracy)+'.keras'
RN.save(model_filename)
print("Saved model as ", model_filename)

# Delete the model to free up resources
del RN

Epoch 1/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 204s 56ms/step - accuracy: 0.8216 - loss: 0.5414 - precision: 0.8911 - recall: 0.7732 - val_accuracy: 0.9228 - val_loss: 0.2744 - val_precision: 0.9288 - val_recall: 0.9190
Epoch 2/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 92s 38ms/step - accuracy: 0.9748 - loss: 0.0744 - precision: 0.9781 - recall: 0.9714 - val_accuracy: 0.9241 - val_loss: 0.2946 - val_precision: 0.9318 - val_recall: 0.9224
Epoch 3/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 91s 38ms/step - accuracy: 0.9836 - loss: 0.0495 - precision: 0.9852 - recall: 0.9820 - val_accuracy: 0.9270 - val_loss: 0.3015 - val_precision: 0.9308 - val_recall: 0.9258
Epoch 4/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 90s 38ms/step - accuracy: 0.9882 - loss: 0.0331 - precision: 0.9891 - recall: 0.9871 - val_accuracy: 0.9203 - val_loss: 0.3252 - val_precision: 0.9247 - val_recall: 0.9174
Epoch 5/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 90s 37ms/step - accuracy: 0.9894 - loss: 0.0318 - precision: 0.9904 - recall: 0.9886 - val_

## EfficientNetB4



In [41]:
def preprocess_images_in_batches_efficientnet(images, batch_size=1024):
    # Placeholder for the preprocessed dataset
    preprocessed_images = np.empty_like(images, dtype=np.float32)

    # Calculate number of batches
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(images))

        # Preprocess the current batch
        batch = images[start:end].astype('float32')  # Ensure float32 for preprocessing
        preprocessed_images[start:end] = preprocess_efficientnet(batch)

        # Free up memory by deleting the batch
        del batch

    return preprocessed_images

In [42]:
# Scale input to EfficientNet-specific format
X_train_EN = preprocess_images_in_batches_efficientnet(X_train)
X_val_EN = preprocess_images_in_batches_efficientnet(X_val)

In [43]:
# Make label tensors one-hot encoded
y_train_EN = to_categorical(y_train)
y_val_EN = to_categorical(y_val)

In [44]:
# Input shape for the model
input_shape_EN = X_train_EN.shape[1:]

# Output shape for the model
output_shape_EN = y_train_EN.shape[1]

print("Input Shape:", input_shape_EN)
print("Output Shape:", output_shape_EN)

Input Shape: (96, 96, 3)
Output Shape: 8


In [45]:
# Set training parameters

epochs_EN = 1000

batch_size_EN = 32

learning_rate_EN = 0.001

print("Epochs:", epochs_EN)
print("Batch Size:", batch_size_EN)
print("Learning Rate:", learning_rate_EN)

Epochs: 1000
Batch Size: 32
Learning Rate: 0.001


In [46]:
# Calculate appropriate class weights to make up for dataset imbalance

classes = np.unique(np.argmax(y_train_EN, axis=1))
class_weights = compute_class_weight('balanced', classes=classes, y=np.argmax(y_train_RN, axis=1))
class_weight_dict_EN = dict(enumerate(class_weights))
print("Class Weights:", class_weight_dict_EN)

Class Weights: {0: 2.166440217391304, 1: 0.6357655502392344, 2: 1.519536213468869, 3: 0.6932608695652174, 4: 2.170372050816697, 5: 1.7231628242074928, 6: 0.5885211614173228, 7: 0.8891263940520446}


In [47]:
# Implementation of EfficientNet, not fine-tunable (only used to select baseline model)

def build_efficientnet(
    input_shape=input_shape_EN,
    output_shape=output_shape_EN,
    learning_rate=learning_rate_EN,
    seed=seed
):
    tf.random.set_seed(seed)

    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    base_model = EfficientNetB4(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = tf.keras.layers.Input(shape=input_shape, name='Input')
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = tf.keras.layers.Dense(128, activation='relu', name='dense_1')(x)
    x = tf.keras.layers.Dropout(0.3, seed=seed, name='dropout_1')(x)
    outputs = tf.keras.layers.Dense(units=output_shape, activation='softmax', name='output')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='EfficientNetB4')

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    mixed_precision_optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

    model.compile(loss='categorical_crossentropy', optimizer=mixed_precision_optimizer, metrics=['accuracy', 'precision', 'recall'])

    return model

In [48]:
EN = build_efficientnet()
EN.summary()

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "EfficientNetB4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 96, 96, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb4 (Functional)          │ (None, 3, 3, 1792)          │      17,673,823 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_avg_pool                      │ (None, 1792)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │         229,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,904,359 (68.30 MB)

 Trainable params: 230,536 (900.53 KB)

 Non-trainable params: 17,673,823 (67.42 MB)

In [49]:
patience_EN = 5

early_stopping_EN = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience_EN,
    restore_best_weights=True
)

# Store the callback in a list
callbacks_EN = [early_stopping_EN]

In [50]:
# EfficientNet training
# Trained on the baseline level of augmentation, mimicking the test we did to choose our baseline model

# Train the model with early stopping callback
history = EN.fit(
    x=X_train_EN,
    y=y_train_EN,
    batch_size=batch_size_EN,
    epochs=epochs_EN,
    validation_data=(X_val_EN, y_val_EN),
    callbacks=callbacks_EN,
    class_weight=class_weight_dict_EN
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

model_filename = 'EffNet_NonFineTunedForShow'+str(final_val_accuracy)+'.keras'
EN.save(model_filename)
print("Saved model as ", model_filename)

# Delete the model to free up resources
del EN

Epoch 1/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 128s 35ms/step - accuracy: 0.6433 - loss: 0.9772 - precision: 0.7791 - recall: 0.4970 - val_accuracy: 0.8171 - val_loss: 0.5261 - val_precision: 0.8681 - val_recall: 0.7567
Epoch 2/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 43s 18ms/step - accuracy: 0.7748 - loss: 0.6108 - precision: 0.8342 - recall: 0.7100 - val_accuracy: 0.8318 - val_loss: 0.4822 - val_precision: 0.8728 - val_recall: 0.7940
Epoch 3/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 42s 18ms/step - accuracy: 0.7973 - loss: 0.5406 - precision: 0.8476 - recall: 0.7465 - val_accuracy: 0.8368 - val_loss: 0.4707 - val_precision: 0.8735 - val_recall: 0.8108
Epoch 4/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 44s 18ms/step - accuracy: 0.8087 - loss: 0.5094 - precision: 0.8534 - recall: 0.7620 - val_accuracy: 0.8473 - val_loss: 0.4330 - val_precision: 0.8813 - val_recall: 0.8188
Epoch 5/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 43s 18ms/step - accuracy: 0.8161 - loss: 0.4827 - precision: 0.8568 - recall: 0.7744 - val_

## MobileNetV3Small


In [31]:
def preprocess_images_mobilenet(images, batch_size=1024):
    # Placeholder for the preprocessed dataset
    preprocessed_images = np.empty_like(images, dtype=np.float32)

    # Calculate number of batches
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(images))

        # Preprocess the current batch
        batch = images[start:end].astype('float32')  # Ensure float32 for preprocessing
        preprocessed_images[start:end] = preprocess_mobilenet(batch)

        # Free up memory by deleting the batch (not strictly necessary in Python)
        del batch

    return preprocessed_images

In [32]:
# Scale input to MobileNet-specific format
X_train_MN = preprocess_images_mobilenet(X_train)
X_val_MN = preprocess_images_mobilenet(X_val)

In [33]:
# Make label tensors one-hot encoded
y_train_MN = to_categorical(y_train)
y_val_MN = to_categorical(y_val)

In [34]:
# Input shape for the model
input_shape_MN = X_train_MN.shape[1:]

# Output shape for the model
output_shape_MN = y_train_MN.shape[1]

print("Input Shape:", input_shape_MN)
print("Output Shape:", output_shape_MN)

Input Shape: (96, 96, 3)
Output Shape: 8


In [35]:
# Set training parameters
epochs_MN = 1000

batch_size_MN = 32

learning_rate_MN = 0.001

# Print the defined parameters
print("Epochs:", epochs_MN)
print("Batch Size:", batch_size_MN)
print("Learning Rare:", learning_rate_MN)

Epochs: 1000
Batch Size: 32
Learning Rare: 0.001


In [36]:
# Calculate appropriate class weights to make up for dataset imbalance
classes = np.unique(np.argmax(y_train_MN, axis=1))
class_weights = compute_class_weight('balanced', classes=classes, y=np.argmax(y_train_MN, axis=1))
class_weight_dict_MN = dict(enumerate(class_weights))
print("Class Weights:", class_weight_dict_MN)

Class Weights: {0: 2.166440217391304, 1: 0.6357655502392344, 2: 1.519536213468869, 3: 0.6932608695652174, 4: 2.170372050816697, 5: 1.7231628242074928, 6: 0.5885211614173228, 7: 0.8891263940520446}


In [37]:
# Implementation of MobileNetV3, not fine-tunable (only used to select baseline model)

def build_mobilenet_v3_small(
    input_shape=input_shape_MN,
    output_shape=output_shape_MN,
    learning_rate=learning_rate_MN,
    seed=seed
):
    tf.random.set_seed(seed)

    # Enable mixed precision
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision policy set to: 'mixed_float16'")

    # Load MobileNetV3Small as the base model
    base_model = MobileNetV3Small(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False

    # Add custom classification head
    inputs = tf.keras.layers.Input(shape=input_shape, name='Input')
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = tf.keras.layers.Dense(128, activation='relu', name='dense_1')(x)
    x = tf.keras.layers.Dropout(0.3, seed=seed, name='dropout_1')(x)
    outputs = tf.keras.layers.Dense(units=output_shape, activation='softmax', name='output')(x)

    # Create the final model
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='MobileNetV3Small')

    # Compile the model with mixed precision optimizer
    adam_optimizer = Adam(learning_rate=learning_rate)
    mixed_precision_optimizer = LossScaleOptimizer(adam_optimizer)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=mixed_precision_optimizer,
        metrics=['accuracy', 'precision', 'recall']
    )

    return model

In [38]:
MN = build_mobilenet_v3_small()
MN.summary()

Mixed precision policy set to: 'mixed_float16'


/usr/local/lib/python3.10/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Model: "MobileNetV3Small"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 96, 96, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MobileNetV3Small (Functional)        │ (None, 3, 3, 576)           │         939,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_avg_pool                      │ (None, 576)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,014,008 (3.87 MB)

 Trainable params: 74,888 (292.53 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [39]:
patience_MN = 5

early_stopping_MN = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience_MN,
    restore_best_weights=True
)

callbacks_MN = [early_stopping_MN]

In [40]:
#MobileNet training
# Trained on the baseline level of augmentation, mimicking the test we did to choose our baseline model

history = MN.fit(
    x=X_train_MN,
    y=y_train_MN,
    batch_size=batch_size_MN,
    epochs=epochs_MN,
    validation_data=(X_val_MN, y_val_MN),
    callbacks=callbacks_MN,
    class_weight=class_weight_dict_MN
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

model_filename = 'MobileNet_NotFineTuned'+str(final_val_accuracy)+'.keras'
MN.save(model_filename)
print("Saved model as ", model_filename)

# Delete the model to free up resources
del MN

Epoch 1/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 48s 14ms/step - accuracy: 0.3191 - loss: 1.6861 - precision: 0.7745 - recall: 0.0549 - val_accuracy: 0.5101 - val_loss: 1.3393 - val_precision: 0.8463 - val_recall: 0.1732
Epoch 2/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.4650 - loss: 1.3454 - precision: 0.7943 - recall: 0.2117 - val_accuracy: 0.5508 - val_loss: 1.2397 - val_precision: 0.8073 - val_recall: 0.2601
Epoch 3/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.5132 - loss: 1.2530 - precision: 0.7757 - recall: 0.2747 - val_accuracy: 0.5831 - val_loss: 1.1643 - val_precision: 0.8709 - val_recall: 0.2999
Epoch 4/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.5434 - loss: 1.1898 - precision: 0.7737 - recall: 0.3165 - val_accuracy: 0.6170 - val_loss: 1.0746 - val_precision: 0.8444 - val_recall: 0.3687
Epoch 5/1000
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.5670 - loss: 1.1370 - precision: 0.7743 - recall: 0.3557 - val_accur

# Preparing for hand-in

In [ ]:
%%writefile model.py
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_mobilenet


def preprocess_images_in_batches_efficientnet(images, batch_size=1024):
    # Placeholder for the preprocessed dataset
    preprocessed_images = np.empty_like(images, dtype=np.float32)

    # Calculate number of batches
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(images))

        # Preprocess the current batch
        batch = images[start:end].astype('float32')  # Ensure float32 for preprocessing
        preprocessed_images[start:end] = preprocess_efficientnet(batch)

        # Free up memory by deleting the batch
        del batch

    return preprocessed_images

def preprocess_images_in_batches_resnet(images, batch_size=1024):
    # Placeholder for the preprocessed dataset
    preprocessed_images = np.empty_like(images, dtype=np.float32)

    # Calculate number of batches
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(images))

        # Preprocess the current batch
        batch = images[start:end].astype('float32')  # Ensure float32 for preprocessing
        preprocessed_images[start:end] = preprocess_resnet(batch)

        # Free up memory by deleting the batch (not strictly necessary in Python)
        del batch

    return preprocessed_images

def preprocess_images_mobilenet(images, batch_size=1024):
    # Placeholder for the preprocessed dataset
    preprocessed_images = np.empty_like(images, dtype=np.float32)

    # Calculate number of batches
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(images))

        # Preprocess the current batch
        batch = images[start:end].astype('float32')  # Ensure float32 for preprocessing
        preprocessed_images[start:end] = preprocess_mobilenet(batch)

        # Free up memory by deleting the batch (not strictly necessary in Python)
        del batch

    return preprocessed_images

class Model:
    def __init__(self):

        self.neural_network = tfk.models.load_model('YOU FORGOT THE FILENAME')
        self.neural_network.trainable = False

    def predict(self, X):
    #NB! Choose correct normalization function depending on model being evaluated!
        X_scaled = preprocess_images_in_batches_resnet(X)
        preds = self.neural_network.predict(X_scaled)

        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)

        return preds


In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py MODEL FILENAME

from google.colab import files
files.download(filename)